In [ ]:
import requests

cookies = {
        'JSESSIONID': 'F5CE3110AA6AE1BFF3B9F746E5574157',
    }

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Referer': 'http://tools.nlp.itu.edu.tr/IsTurkish',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0',
}

In [ ]:
from bs4 import BeautifulSoup

def isTurkish(word):
    
    # Formu POST ile gönder
    form_data = {
        'input': word  # Kelimeyi input alanına gönderiyoruz
    }

    response = requests.post('http://tools.nlp.itu.edu.tr/IsTurkish', cookies=cookies, headers=headers, data=form_data, verify=False)

    # Sayfa içeriğini BeautifulSoup ile işle
    soup = BeautifulSoup(response.text, 'html.parser')

    # Sayfa içeriğinde yer alan 'output' alanını al
    result = soup.find('textarea', {'name': 'output'})

    if result:
        # 'output' alanındaki sonucu sadece true/false olarak döndür
        result_text = result.get_text().strip().lower()
        if "true" in result_text:
            return True
        else:
            return False

In [ ]:
import json

def process_words(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    new_roots = {}
    
    for word, word_id in data.items():

        # Fiil olup olmadığını kontrol et
        potential_verb = word + 'mak' if word[-1] not in ['e'] else word + 'mek'

        if isTurkish(potential_verb):
            # Fiil ise ünlü daralmasını uygula
            
            if(word in ['de','ye','ne']):
                if word == 'de':
                    new_roots['di'] = word_id
                elif word == 'ye':
                    new_roots['yi'] = word_id
                elif word == 'ne':
                    new_roots['ni'] = word_id

            elif(word[-1] in 'ae'):
                if word[-1] == 'a' and ('a' in word[:-1] or 'ı' in word[:-1]):
                    new_word = word[:-1] + 'ı'  # 'a' -> 'ı'
                    new_roots[new_word] = word_id
                elif word[-1] == 'a' and ('a' not in word[:-1] or 'u' in word[:-1]):
                    new_word = word[:-1] + 'u'  # 'a' -> 'u'
                    new_roots[new_word] = word_id
                elif word[-1] == 'e' and ('e' in word[:-1] or 'i' in word[:-1]):
                    new_word = word[:-1] + 'i'  # 'e' -> 'i'
                    new_roots[new_word] = word_id
                else:
                    new_word = word[:-1] + 'ü'  # 'e' -> 'ü'
                    new_roots[new_word] = word_id
    
    # Orijinal kelimeler ve yeni kökleri birleştir
    data.update(new_roots)
    
    # Güncellenmiş veriyi JSON dosyasına yaz
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    return new_roots


In [ ]:
daralmaliListe = process_words('/Users/yusuf/Desktop/tokenizer/turkish_tokenizer/kokler_v05.json')